In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch

import warnings
warnings.filterwarnings("ignore") 

import sys
sys.path.append('../src')

In [3]:
# Names of the columns that are discrete
discrete_columns = [
    'State', 'Zip', 'Bank',
    'BankState', 'ApprovalFY',
    'NewExist', 'UrbanRural',
    'RevLineCr', 'LowDoc', 'MIS_Status',
]

In [4]:
df = pd.read_csv('../data/loan/loan_train.csv')
df.rename(lambda x: x.strip(), axis='columns', inplace=True)

In [5]:
df.shape

(120000, 21)

In [6]:
df.head()

,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,...,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,PA,18036,WELLS FARGO BANK NATL ASSOC,PA,0,1998,180,8,2.0,0,...,1,0,N,N,287368.0,0.0,P I F,0.0,360000.0,270000.0
1,WI,54487,CAPITAL ONE NATL ASSOC,VA,443120,2004,36,2,1.0,0,...,1,1,N,N,50000.0,0.0,CHGOFF,30598.0,50000.0,25000.0
2,PA,15024,"PNC BANK, NATIONAL ASSOCIATION",DE,323119,2006,60,7,2.0,0,...,0,2,N,N,27900.0,0.0,P I F,0.0,27900.0,13950.0
3,MO,64424,BANK NORTHWEST,MO,811111,1998,156,4,1.0,0,...,1,0,N,N,106500.0,0.0,P I F,0.0,106500.0,79875.0
4,IN,46203,PREMIER CAPITAL CORPORATION,IN,0,1983,240,95,1.0,0,...,0,0,N,N,189000.0,0.0,P I F,0.0,189000.0,189000.0


In [7]:
from synthesizers.metgan import *
metgan = MeTGANSynthesizer(embedding_dim=64,generator_dim=(256,256),discriminator_dim=(256,256),batch_size=500,discriminator_steps=1,verbose=True)

In [8]:
metgan.fit(df, discrete_columns, epochs=3)  # Epochs = 300 used in the actual experiments

Data_dim_p:  210940
Epoch 1, Loss G:  2.6596,Loss D: -0.1834
Epoch 2, Loss G:  1.6420,Loss D:  0.0028
Epoch 3, Loss G:  1.6659,Loss D: -0.0099


In [10]:
syn_data = metgan.sample(150000)  # Size of complete Loan dataset

In [11]:
syn_data.head()

,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,...,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,MO,44236,CALIFORNIA BANK & TRUST,IL,450609,2006,42,2,1.0,0,...,-91,1,0,N,116658.137823,7.685720,P I F,260.758925,33911.533568,82407.400680
1,CA,91355,MANUFACTURERS & TRADERS TR CO,CA,12741,2006,104,7,2.0,0,...,-92,1,N,N,69120.114570,-2.214033,CHGOFF,503.337629,14765.111730,42281.932928
2,MI,21047,HSBC BK USA NATL ASSOC,KS,726827,1997,83,1,2.0,0,...,11,0,N,N,529896.522159,-4.407408,CHGOFF,49.330557,4645.531629,21665.571640
3,FL,20904,BBCN BANK,IL,583305,2006,93,7,1.0,0,...,-53,0,N,N,43519.217802,-16.656495,P I F,229.215212,-8799.841903,21863.471770
4,AL,95130,NEVADA ST. DEVEL CORP,MS,326010,2006,177,5,1.0,0,...,18,2,0,N,137672.653675,-2.322799,P I F,122.356600,94032.115725,202206.150182


In [12]:
syn_data.to_csv('../syn/metgan-loan-3epochs.csv',index=False)